In [66]:
import numpy as np
import pandas as pd
import requests
import sqlite3
from pathlib import Path


In [34]:
polling = pd.read_csv("../president_primary_polls.csv")


In [35]:
polling_subset = polling[polling["party"] == "DEM"]
polling["create_date"] = pd.to_datetime(polling["created_at"], infer_datetime_format=True)
polling["create_date"]
polling["create_week"] = (polling['create_date'] - pd.to_timedelta(polling['create_date'].dt.dayofweek, unit='d')
                          + np.timedelta64(7, 'D')).dt.normalize()                                                             
polling_subset = polling[polling["party"] == "DEM"]
df_pivot = polling[polling["party"] == "DEM"].pivot_table(values="pct", index="create_week", columns="answer",
                                                          aggfunc=np.mean).fillna(0).iloc[-7:]
candidates = polling_subset["answer"].unique()


In [7]:
can_selected = []
for can in candidates:
    if (df_pivot[can] == 0).mean() < 0.5 and df_pivot[can].mean() > 0.1:
        print(can)
        can_selected.append(can)


Booker
Biden
Sanders
Warren
Buttigieg
Harris
Yang
Bloomberg
Castro
Klobuchar
Gabbard
Steyer
Delaney
Sestak
Williamson
Bullock
Bennet
O'Rourke
Ryan


In [14]:
print('Beginning file download with requests')

for can in can_selected:
    url = 'https://api.gdeltproject.org/api/v2/tv/tv?query="' + can + '"%20market:%22National%22&mode=timelinevol&format' \
                                                                      '=csv&datanorm=perc&dateres=week&timelinesmooth=0' \
                                                                      '&datacomb=sep' \
                                                                      '&last24=yes&timezoom=yes '
    r = requests.get(url)

    with open('../candidates_coverage/' + can + '.csv', 'wb') as f:
        f.write(r.content)

    print(url)
    print(r.status_code)
    print(r.headers['content-type'])
    print(r.encoding)


Beginning file download with requests


https://api.gdeltproject.org/api/v2/tv/tv?query="Booker"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8


https://api.gdeltproject.org/api/v2/tv/tv?query="Biden"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8


https://api.gdeltproject.org/api/v2/tv/tv?query="Sanders"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8


https://api.gdeltproject.org/api/v2/tv/tv?query="Warren"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8


...

https://api.gdeltproject.org/api/v2/tv/tv?query="Williamson"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8


https://api.gdeltproject.org/api/v2/tv/tv?query="Bullock"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8


https://api.gdeltproject.org/api/v2/tv/tv?query="Bennet"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8


https://api.gdeltproject.org/api/v2/tv/tv?query="O'Rourke"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8


https://api.gdeltproject.org/api/v2/tv/tv?query="Ryan"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8


In [73]:
source_files = sorted(Path('../candidates_coverage').glob('*.csv'))

dataframes = []
for file in source_files:
    df = pd.read_csv(file)
    df['Candidate'] = file.name.replace(".csv", "")
    dataframes.append(df)

media_coverage = pd.concat(dataframes, sort=False)
media_coverage.rename(columns={(media_coverage.columns[0]): 'Date'}, inplace=True)
media_coverage['Date'] = pd.to_datetime(media_coverage['Date'])
media_coverage.to_csv('C:\\Users\\User\\PycharmProjects\\WebDB\\polls\\media_coverage.csv')


In [ ]:
polling_means = polling_subset.groupby(['answer', 'create_week'])['pct'].mean().reset_index()
conn = sqlite3.connect(':memory:')
media_coverage.to_sql('media_coverage', conn, index=False)
polling_means.to_sql('polling_means', conn, index=False)
qry = '''
    SELECT *
    FROM
        polling_means JOIN media_coverage ON
        Candidate = answer
        AND create_week BETWEEN Date AND date(Date, '+7 day')
    '''
corr_data = pd.read_sql_query(qry, conn)
corr_data.to_csv('../polls/corr_data.csv')
